In [79]:
import pandas as pd
import numpy as np
from datetime import datetime 
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
import os
import sys 

import cx_Oracle
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

import sqlite3

# Parameter variable

In [93]:
source_name=''
source_name="yip_invoice_monthly" # df/csv

#source_name='yip_po_listing'
#source_name="yip_ar_receipt"  # df/csv
# source_name='yip_ap_payment' # csv
# source_name="yip_pj_status" # csv
# source_name='yip_gl_account'

# set True whatever , you want to reload all items
isLoadingAllItems=False
is_py=False

init_date_query='2020-01-01 00:00:00'
#init_date_query='2023-04-01 00:00:00'


In [94]:
if is_py:
    press_Y=''
    ok=False

    if len(sys.argv) > 1:
        source_name=sys.argv[1]

        if sys.argv[2]=='0':
         isLoadingAllItems=False
        elif sys.argv[2]=='1'  :
         isLoadingAllItems=True
        else:
            raise Exception("isLoadingAllItems 1=True | 0=False")

        ok=True 

    else:
        print("Enter the following input: ")
        source_name = input("View Table Name : ")
        source_name=source_name.lower()
        load_option= int(input("Loading All Data option (1=True | 0=False): "))
        if load_option==0:
         isLoadingAllItems=False
        elif load_option==1  :
         isLoadingAllItems=True
        else:
            raise Exception("isLoadingAllItems 1=True | 0=False")  

        print(f"Confirm to Load view = {source_name} and Load All Data= {isLoadingAllItems}")
        press_Y=input(f"Press Y=True But any key=False : ") 
        if press_Y=='Y':
         ok=True

    if ok==False:
        print("No any action")
        quit()


# Init Variable

In [95]:
listError=[]

projectId='mismgntdata-bigquery'
region='asia-southeast1'
dataset_id='MIS_BI_DW'
table_id = f"{projectId}.{dataset_id}.{source_name}"

_ip='172.30.57.10' #'172.30.57.10'
_hostname='YIPGERP'
_port=1521
_servicename='PROD'
_username='yipgbi'
_password='yipgbi'


host = 'mail.yipintsoi.com'
port=  25
sender="mis-bi-service@yipintsoigroup.com"
receivers=['pongthorn.sa@yipintsoi.com']
#receivers=['pongthorn.sa@yipintsoi.com','mis-bi-service@yipintsoigroup.com']

data_base_file=r'D:\ETL_Orable_To_BQ\etl_web_admin\etl_config_transaction.db'
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

json_credential_file=r'C:\Windows\mismgntdata-bigquery--bq-loader-34713c332847.json'

temp_path=f'temp/{source_name}.csv'


start_date_query=''
updateCol='last_update_date'

In [96]:
dt_imported=datetime.now()
dtStr_imported=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
#dtStr_imported='2023-05-23 23:00:00'

dt_imported=datetime.strptime(dtStr_imported,"%Y-%m-%d %H:%M:%S")

print(dtStr_imported)
print(dt_imported)

2023-06-12 23:21:27
2023-06-12 23:21:27


# Email Nofification &  Manage Log Error Message

In [97]:
import  smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def sendMailForError(errorHtml):
    message = MIMEMultipart("alternative")
    message["Subject"] = f"MIS-BI : ETL-Error on {source_name} at {dtStr_imported}"
    message["From"] = sender
    message["To"] = ','.join(receivers)

    html =errorHtml 

    part_html = MIMEText(html, "html")
    message.attach(part_html)

    try:

        with smtplib.SMTP(host,port) as mail_server:
            #mail_server.login(login, password)
            mail_server.sendmail(sender, receivers, message.as_string())
            print("Successfully sent email")

    except (gaierror, ConnectionRefusedError):
       msg='Failed to connect to the server. Bad connection settings?'
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg]) 
       logErrorMessage(listError)
    except smtplib.SMTPServerDisconnected:
       msg='Failed to connect to the server. Wrong user/password?'
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg]) 
       logErrorMessage(listError)
    except smtplib.SMTPException as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
       logErrorMessage(listError)

    return True
    

In [98]:
def logErrorMessage(errorList,raise_ex=True):
    
    def add_error_to_file(error_des):
        f = open(r'log_error.txt', 'a')
        error_str = f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}|{repr(error_des)}\n'

        f.write(error_str)
        f.close()
        print(error_str)
        raise Exception(error_str)
    
    def add_logError(recordList):
        try:
            sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
            cursor = sqliteConnection.cursor()
            sqlite_insert_query = """
            INSERT INTO log_error
            (error_datetime,etl_datetime, data_source_id,message)  VALUES (?,?,?,?);
             """
            cursor.executemany(sqlite_insert_query, recordList)
            print("Done Log Error")
            sqliteConnection.commit()
            cursor.close()
            
        except Exception as e:
            msg=f"{data_base_file} error : {str(e)}"
            listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
            add_error_to_file(msg)
        finally:
            if sqliteConnection:
                sqliteConnection.close()
            
    if len(errorList)>0:
        error_message=f"{source_name} ETL at {dt_imported} raise some errors."
        print(error_message)
        
        dfError=pd.DataFrame(data=errorList,columns=["error_datetime","etl_datetime","data_source_id","message"])
        print(f"Total log error={len(dfError)}")
        print(dfError)
        
        
        recordError=dfError.to_records(index=False)
        add_logError(recordError)
        
        emailResult=sendMailForError(dfError.to_html(index=False))
        
        error_message=f"{source_name} ETL at {dt_imported} raise some errors."
        
        if raise_ex==True:
         raise  Exception(error_message)
    

# Get & Set Oracle ViewName and other configuration data

In [99]:
# get data from data_source
def get_ds(data_source_name):
   try: 
        conn = sqlite3.connect(os.path.abspath(data_base_file))
        sql_ds=f"""select * from data_source where id='{data_source_name}'  """
        print(sql_ds)
        df_item=pd.read_sql_query(sql_ds, conn)
        if df_item.empty==False:
           return df_item.iloc[0,:]
        else:
           return None
   except Exception as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
       logErrorMessage(listError)
    
ds_item=get_ds(source_name)
ds_item

select * from data_source where id='yip_invoice_monthly'  


id                                  yip_invoice_monthly
first_load_col                                  gl_date
partition_date_col                              gl_date
partition_date_type                               MONTH
cluster_col_list       INVOICE_DATE ,gl_date ,dept_name
date_col_list                      gl_date,invoice_date
Name: 0, dtype: object

In [100]:
if ds_item is not None:
    print("Load data source config data")

    colFirstLoad=ds_item['first_load_col']
    colFirstLoad=colFirstLoad.strip().lower()
    print(f"Date Column as condition to load at first = {colFirstLoad}")

    partitionCol=ds_item['partition_date_col']  # required DateTime Type
    partitionCol=partitionCol.strip().lower()
    if   ds_item['partition_date_type']=="DAY":
     partitionType=bigquery.TimePartitioningType.DAY
    elif ds_item['partition_date_type']=="MONTH":
     partitionType=bigquery.TimePartitioningType.MONTH    
    elif ds_item['partition_date_type']=="YEAR":
     partitionType=bigquery.TimePartitioningType.YEAR   
    else:
     partitionType=bigquery.TimePartitioningType.DAY
    
    print(f"Partition : {partitionCol} - {partitionType}")
    
        
    if ds_item['cluster_col_list']=='':
     clusterCols=[]   
     print(f"{clusterCols} (No cluster cols)")   
     
    else:
     clusterCols=  ds_item['cluster_col_list'].split(',') 
     clusterCols = list(map(str.strip,clusterCols))
     clusterCols = list(map(str.lower,clusterCols))   
     print(f"Cluster List: {clusterCols}")

    if ds_item['date_col_list']=='':
     dateCols=[]   
     print(f"{dateCols} (No Date cols)")   
     
    else:
     dateCols=  ds_item['date_col_list'].split(',') 
     dateCols = list(map(str.strip,dateCols))
     dateCols = list(map(str.lower,dateCols))      
     print(f"Date Cols: {dateCols}")

else:
   listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,f"Not found view {source_name} in data_source table"])
   logErrorMessage(listError)

Load data source config data
Date Column as condition to load at first = gl_date
Partition : gl_date - MONTH
Cluster List: ['invoice_date', 'gl_date', 'dept_name']
Date Cols: ['gl_date', 'invoice_date']


# List Last ETL Transacton by Datasource Name
### Get last etl of the specific view to perform incremental update

In [101]:
def get_last_etl_by_ds(data_source):
   try: 
    conn = sqlite3.connect(os.path.abspath(data_base_file))
    sql_last_etl=f"""select etl_datetime,data_source_id from etl_transaction where data_source_id='{data_source}' 
    order by etl_datetime desc limit 1
    """
    print(sql_last_etl)
    df_item=pd.read_sql_query(sql_last_etl, conn)
    print(df_item)
    return df_item
    
   except Exception as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
       logErrorMessage(listError)

if isLoadingAllItems==False:
    dfLastETL=get_last_etl_by_ds(source_name)
    if dfLastETL.empty==False:
      start_date_query=dfLastETL.iloc[0,0]
      print(f"Start Import on update_at of last ETL date :  {start_date_query}" ) 
    else:
       isLoadingAllItems=True 
       start_date_query=init_date_query
       print(f"No etl transaction , we will get started with importing all items from :  {start_date_query}" ) 
else:
   start_date_query=init_date_query
   print(f"Reload all data:  {start_date_query}" ) 
        


select etl_datetime,data_source_id from etl_transaction where data_source_id='yip_invoice_monthly' 
    order by etl_datetime desc limit 1
    
Empty DataFrame
Columns: [etl_datetime, data_source_id]
Index: []
No etl transaction , we will get started with importing all items from :  2020-01-01 00:00:00


# Load data from Oracel  as DataFrame 

In [102]:
def loadData(isReLoadAll):
    print(f"isReLoadAll=={isReLoadAll}")  
    try:
       engine = sqlalchemy.create_engine(f"oracle+cx_oracle://{_username}:{_password}@{_ip}:{_port}/?service_name={_servicename}")
       if isReLoadAll==True:
          
         sql=f"""select * from {source_name}   
           where  {colFirstLoad}>=to_date('{start_date_query}','yyyy-mm-dd hh24:mi:ss') 
           """    
       else:    
           sql =f"""select * from {source_name}  
           where  {updateCol}>=to_date('{start_date_query}','yyyy-mm-dd hh24:mi:ss') """
            
       print(sql)
       
       dfAll = pd.read_sql(sql, engine)
    
       return dfAll 
    except Exception as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
       logErrorMessage(listError)
    

dfAll=loadData(isLoadingAllItems)


isReLoadAll==True
select * from yip_invoice_monthly   
           where  gl_date>=to_date('2020-01-01 00:00:00','yyyy-mm-dd hh24:mi:ss') 
           


# Test Data

In [103]:
# dfAll['last_update_date'].unique()

In [104]:
# dfAll=dfAll.drop(columns=['line_description8','line_description9'  ,'dept_code'])
# _dateColTest='last_update_date'
# _dateValueTest='2023-06-09 22:45:09'
# dfAll=dfAll.query(f"{_dateColTest}<@_dateValueTest")
# dt_imported=datetime.strptime(_dateValueTest,'%Y-%m-%d %H:%M:%S')
# dtStr_imported=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
#=============================================================================


In [105]:
dfAll['ImportedAt']=dt_imported 

listColDF=dfAll.columns.tolist()
print("List columns of DF")
print(listColDF)

print(dfAll.info())
dfAll.head()

List columns of DF
['customer_trx_id', 'org_id', 'gl_date', 'invoice_date', 'dept_name', 'dept_code', 'invoice_number', 'tax_invoice_number', 'so', 'project', 'rev_con', 'cn_number', 'cn_comment', 'invoice_type', 'invoice_type_description', 'cust_code', 'cust_name', 'industry_new', 'sector', 'abbrevation', 'sale_code', 'sale_name', 'invoice_amount', 'total_cost', 'margin_amount', 'margin_percent', 'salesforce_opp', 'line_description1', 'line_description2', 'line_description3', 'line_description4', 'line_description5', 'line_description6', 'line_description7', 'line_description8', 'line_description9', 'creation_date', 'created_by', 'last_update_date', 'last_updated_by', 'ImportedAt']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22412 entries, 0 to 22411
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   customer_trx_id           22412 non-null  int64         
 1   org_id  

,customer_trx_id,org_id,gl_date,invoice_date,dept_name,dept_code,invoice_number,tax_invoice_number,so,project,...,line_description5,line_description6,line_description7,line_description8,line_description9,creation_date,created_by,last_update_date,last_updated_by,ImportedAt
0,791916,81,2020-10-14,2020-10-14,MGT,9996,312001180,312001180,None,None,...,None,None,None,None,None,2020-10-14 00:00:00,1073,2023-06-08 13:55:03,1073,2023-06-12 23:21:27
1,835001,81,2022-03-03,2022-03-03,MGT,9996,312200377,312200377,None,None,...,None,None,None,None,None,2022-03-03 00:00:00,1073,2023-06-08 13:55:03,1073,2023-06-12 23:21:27
2,876304,81,2023-01-05,2023-01-05,MGT,9996,312300076,None,None,None,...,None,None,None,None,None,2023-01-05 09:59:05,1073,2023-06-08 13:55:03,1073,2023-06-12 23:21:27
3,793721,81,2020-11-02,2020-11-02,MGT,9996,312001420,312001420,None,None,...,None,None,None,None,None,2020-11-02 00:00:00,1073,2023-06-08 13:55:03,1152,2023-06-12 23:21:27
4,830290,81,2022-01-06,2022-01-06,MGT,9996,312200037,None,None,None,...,None,None,None,None,None,2022-01-06 09:27:20,1073,2023-06-08 13:55:03,1073,2023-06-12 23:21:27


In [39]:
listColAdminConfig=[colFirstLoad,partitionCol]
if len(clusterCols)>0:
 listColAdminConfig.extend(clusterCols)
if len(dateCols)>0:
 listColAdminConfig.extend(dateCols)
listColAdminConfig=list(dict.fromkeys(listColAdminConfig))

checkSomeNotExistingDF = [ col   for col in listColAdminConfig if col not in listColDF ]
if len(checkSomeNotExistingDF)>0:
    msg=f"Some columns in data source are not in dataframe from Oracel View = {checkSomeNotExistingDF }"
    listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg])
    logErrorMessage(listError)
else:
    print(f"{listColAdminConfig} is in dataframe from Oracel View")

['gl_date', 'invoice_date', 'dept_name'] is in dataframe from Oracel View


In [40]:
if dfAll.empty:
    print("No row to import to BQ")
    exit()

# BigQuery

In [41]:
credentials = service_account.Credentials.from_service_account_file(json_credential_file)
client = bigquery.Client(credentials=credentials, project=projectId)

## Creaste bigquery schema from dataframe

In [42]:
# schema = [
# bigquery.SchemaField("CUSTOMER_TRX_ID", "INTEGER", mode="NULLABLE"),
# bigquery.SchemaField("GL_DATE", "DATE", mode="NULLABLE"),
# bigquery.SchemaField("DEPT_NAME", "STRING", mode="NULLABLE"),      
# bigquery.SchemaField("INVOICE_AMOUNT", "FLOAT", mode="NULLABLE"),    
# bigquery.SchemaField("LAST_UPDATE_DATE", "TIMESTAMP", mode="NULLABLE"),
# ]
#https://cloud.google.com/bigquery/docs/schemas
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html
schema = []
srCols=dfAll.dtypes
try:
    for name, type_name in srCols.items():
        # print(name,type_name)
        if str(type_name) in ['int32','int64']:
          schema.append(bigquery.SchemaField(name, "INTEGER", mode="NULLABLE"))
        elif str(type_name) =='float64':
          schema.append(bigquery.SchemaField(name, "FLOAT", mode="NULLABLE"))
        elif str(type_name) =='datetime64[ns]':
          if name in   dateCols:
             schema.append(bigquery.SchemaField(name,  "DATE", mode="NULLABLE"))
          else:
             schema.append(bigquery.SchemaField(name,  "DATETIME", mode="NULLABLE"))
        elif str(type_name) == 'bool':
             schema.append(bigquery.SchemaField(name,  "BOOL", mode="NULLABLE"))
        else: # if not found type , it will be converted to STRING
           schema.append(bigquery.SchemaField(name,  "STRING", mode="NULLABLE")) 
        
    # if  len(schema)!=len(listColDF):
    #    listFieldBQError=[field.name for field in schema]
    #    intersec_DF_BQ = [set(listColDF).symmetric_difference(set(listFieldBQError))]
    #    raise Exception(f"{len(schema)}!={len(listColDF)} , {intersec_DF_BQ} in Dataframe can not be converted to Bigquery Data type")

except Exception as e:
   listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
   logErrorMessage(listError)
print("Total number of field of the schema :",len(schema))    
print(schema) 

Total number of field of the schema : 38
[SchemaField('customer_trx_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('org_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('gl_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('invoice_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('dept_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('invoice_number', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('tax_invoice_number', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('so', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('project', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('rev_con', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('cn_number', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('cn_comment', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('invoice_type', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('invoice_type_description',

## Check whether dataframe and bigquery schema are the same

## Check Existing DataSet and Table

In [43]:
# dataset
try:
    dataset = client.get_dataset(f"{projectId}.{dataset_id}")
    print("Dataset {} already exists".format(dataset_id))
except Exception as ex:
    msg=f"Dataset {dataset_id} is not found"
    listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg]) 
    logErrorMessage(listError)

Dataset MIS_BI_DW already exists


In [44]:
def create_table():
    try:  
        table = bigquery.Table(table_id,schema=schema)
        if  partitionCol!="":
         table.time_partitioning = bigquery.TimePartitioning(
         type_=partitionType,field=partitionCol)

        if len(clusterCols)>0:
         table.clustering_fields = clusterCols

        table = client.create_table(table) 
        print(
            "Created new table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
        )
    except Exception as e:
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
        logErrorMessage(listError)   

In [45]:
def check_table():
    try:
        client.get_table(table_id)
        return True
    except NotFound:
        return False

In [46]:
# def check_same_schema(listFieldBQ,partitionNameBQ,partitionTypeBQ,clusterBQ,dateTypeBQ):
def check_same_schema():
    print("===============================================================================================")
    print("Check every columns name and partition cluster and date type column on table against dataframe")
    def find_difference(dfX,bqX):
        intersec_DF_BQ = [set(dfX).symmetric_difference(set(bqX))]
        list_DF_BQ=[]
        if len(intersec_DF_BQ)>0:
         for item in intersec_DF_BQ:
            list_DF_BQ=list_DF_BQ+list(item)
        return list_DF_BQ 
        
    listColumnX=find_difference(listColDF,listFieldBQ)
    if len(listColumnX)>0:
        e=f"Columns: {listColumnX} are NOT THE SAME on BigQuery and ViewTable {source_name} of Oracle"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("All Fields on BQ and DF are ok.")
        
        
    # PartitionName
    if partitionNameBQ!=partitionCol:
        e=f"Partition Column :{partitionNameBQ} in BQ is NOT THE SAME as {partitionCol} defined on ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("Partition Name Fields on BQ and DF is ok.")    
        

    # # PartitionDateType
    if partitionTypeBQ!=partitionType:
        e=f"Partition Date Type :{partitionTypeBQ} in BQ is NOT THE SAME as {partitionType} defined on ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    
    # Cluster List
    listClusterX=find_difference(clusterCols,clusterBQ)
    if len( listClusterX)>0:
        e=f"Cluster columns : {listClusterX} are NOT THE SAME on BigQuery and ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("All Cluster on BQ and DF are ok.")
    
    # Date Type List
    
        # Cluster List
    listDateColX=find_difference(dateCols,dateTypeBQ)
    if len( listDateColX)>0:
        e=f"Date columns : {listDateColX} are NOT THE SAME on BigQuery and ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("All Date Column on BQ and DF are ok.")

    if len(listError)>0:
        logErrorMessage(listError)
        
# delete table and set isLoading=True to load all data
        
#       isLoadingAllItems=True
#       start_date_query=init_date_query

#       print("ReLoad Data due to something in schema changed")  
#       dfAll=loadData(isLoadingAllItems)
#       print(dfAll.info())  

#       print("Delete table and re-create new one.")
#       client.delete_table(table_id, not_found_ok=True)  
#       create_table()
 
  

In [47]:
if check_table():    
    table=client.get_table(table_id)
    print("Table {} already exists.".format(table_id))
    
    listFieldBQ=[field.name for field in table.schema]
    
    # required field
    partitionNameBQ=table.time_partitioning.field
    partitionTypeBQ=table.partitioning_type

    clusterBQ=table.clustering_fields
    if clusterBQ is None : clusterBQ =[]
        
    dateTypeBQ=[field.name for field in table.schema if field.field_type=='DATE']
    
    
    print(f"All Fields : {listFieldBQ}")
    print(f"Partiton Field&Type: {partitionNameBQ} - {partitionTypeBQ}")
    print(f"Cluster Field List: {clusterBQ}")
    print(f"Date Field List: {dateTypeBQ}")
    
    check_same_schema()
else:
    create_table()
        


Table mismgntdata-bigquery.MIS_BI_DW.yip_invoice_monthly already exists.
All Fields : ['customer_trx_id', 'org_id', 'gl_date', 'invoice_date', 'dept_name', 'dept_code', 'invoice_number', 'tax_invoice_number', 'so', 'project', 'rev_con', 'cn_number', 'cn_comment', 'invoice_type', 'invoice_type_description', 'cust_code', 'cust_name', 'industry_new', 'sector', 'abbrevation', 'sale_code', 'sale_name', 'invoice_amount', 'total_cost', 'margin_amount', 'margin_percent', 'salesforce_opp', 'line_description1', 'line_description2', 'line_description3', 'line_description4', 'line_description5', 'line_description6', 'line_description7', 'line_description8', 'line_description9', 'creation_date', 'created_by', 'last_update_date', 'last_updated_by', 'ImportedAt']
Partiton Field&Type: gl_date - MONTH
Cluster Field List: ['invoice_date', 'gl_date', 'dept_name']
Date Field List: ['gl_date', 'invoice_date']
Check every columns name and partition cluster and date type column on table against dataframe
Par

Exception: yip_invoice_monthly ETL at 2023-06-12 23:03:25 raise some errors.

# To load data into BQ , technically you need  to save it as CSV file first 

In [329]:
# dfAll=dfAll.iloc[:45501,:]  poListing
# #dfAll=dfAll.drop(45502) # error  # for po listing

In [330]:
loading_from='csv' # cvs/dataframe
no_rows=len(dfAll)
no_cols=len(dfAll.columns)
if dfAll.empty==False and loading_from=='csv':
    print("Save dataframe to csv as source first")
    dfAll.to_csv (temp_path,index=False)
print(f"{no_rows} rows and {no_cols} are about to be imported to BQ by {loading_from}")

Save dataframe to csv as source first
1196 rows and 41 are about to be imported to BQ by csv


# Load data from CSV file to BQ

In [331]:
# cannot auto detect because some column , there are Y,N,R  For R BQ is known as Bool
#https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

def collectBQError(x_job):
 if x_job.errors is not None:
    for error in x_job.errors:
      msg=f"{error['reason']} - {error['message']}"
      listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg])
    if   len(logErrorMessage)>0:
     logErrorMessage(listError,False)

try:
    print(f"Load bulk data from {loading_from}")
    if loading_from=='csv' :
        
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1,
            schema=schema,autodetect=False,
            max_bad_records=(no_rows-1),
            # autodetect=True,
            write_disposition="WRITE_APPEND",
            )
        with open(temp_path, "rb") as source_file:
            job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    else :
        # job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")  # ok for POs Listing
        
        job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND",schema=schema)
        job = client.load_table_from_dataframe(dfAll, table_id, job_config=job_config)  


    result_job=job.result()  # Waits for the job to complete.
    
    # error but continue
    collectBQError(job)   
    
    print(f"Import data from {source_name} on Oracle into BQ successfully.")
   


except Exception as e:
  print("BQ Error and Raise Exception")  
  collectBQError(job)
    

Load bulk data from csv
Import data from yip_invoice_monthly on Oracle into BQ successfully.


# Create Transation and delete csv file

In [332]:

#Addtional Try Error    
def insertETLTrans(recordList):
    try:
        sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
        cursor = sqliteConnection.cursor()
        sqlite_insert_query = """
        INSERT INTO etl_transaction
        (etl_datetime, data_source_id,no_rows,is_load_all)  VALUES (?,?,?,?);
         """

        cursor.executemany(sqlite_insert_query, recordList)
        print("Done ETL Trasaction")
        sqliteConnection.commit()
        cursor.close()

    except Exception as e:
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
        logErrorMessage(listError)
        print("Failed to insert etl_transaction table", str(e))
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            


if isLoadingAllItems==True:
    is_load_all=1
else:
    is_load_all=0

dfETFTran=pd.DataFrame.from_records([{'etl_datetime':dtStr_imported,'data_source_id':source_name,'no_rows':no_rows,'is_load_all':is_load_all}])
recordsToInsert=list(dfETFTran.to_records(index=False))
insertETLTrans(recordsToInsert)

Done ETL Trasaction


In [333]:
#Addtional Try Error
try:
    if os.path.exists(temp_path):
      os.remove(temp_path)
      print(f"Deleted {temp_path}")
    
except Exception as e:
    listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    logErrorMessage(listError)

Deleted temp/yip_invoice_monthly.csv
